## Exploring metadata 

Metadata isn't static. The metadata we have got via the ALTO XML has sometimes been superseded by new metadata for an item. There is also some additional metadata which we want to include in our jsonl files. We'll use an exported metadata csv to update existing metadata and add some additional fields that weren't there already. Before we do this we should look at what types of metadata we might want to add in addition to our current set of metadata. In particular we might also want to see how often values are missing from the metadata. If values are missing too often it's not going to be very useful to include them in our new dataset. 

This notebook explores the metadata and decides:

- what metadata we want to include in our dataset
- how we might want to store this metadata in our json 
- what possible processing/cleaning steps might need to take place with the data. 

### imports

In [367]:
import json
import pandas as pd

In [368]:
df = pd.read_excel("MS digitised books 2021-01-09.xlsx")

In [369]:
df

,BL record ID,Type of resource,BNB number,ISBN,Name,Dates associated with name,Type of name,Role,All names,Title,...,Publisher,Date of publication,Edition,Physical description,Dewey classification,BL shelfmark,Topics,Genre,Languages,Notes
0,14602826,Monograph,NaN,NaN,"Yearsley, Ann",1753-1806,person,NaN,"More, Hannah, 1745-1833 [person] ; Yearsley, A...",Poems on several occasions [With a prefatory l...,...,NaN,1786,Fourth edition MANUSCRIPT note,NaN,NaN,Digital Store 11644.d.32,NaN,NaN,English,NaN
1,14602830,Monograph,NaN,NaN,"A, T.",NaN,person,NaN,"Oldham, John, 1653-1683 [person] ; A, T. [person]",A Satyr against Vertue. (A poem: supposed to b...,...,NaN,1679,NaN,15 pages (4°),NaN,Digital Store 11602.ee.10. (2.),NaN,NaN,English,NaN
2,14602831,Monograph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Aeronaut, a poem; founded almost entirely,...",...,Richard Milliken,1816,NaN,17 pages (8°),NaN,Digital Store 992.i.12. (3.),Dublin (Ireland),NaN,English,NaN
3,14602832,Monograph,NaN,NaN,"Albert, Prince Consort, consort of Victoria, Q...",1819-1861,person,NaN,"Plimsoll, Joseph [person] ; Albert, Prince Con...","The Prince Albert, a poem [By Joseph Plimsoll.]",...,W. Cann,1868,NaN,16 pages (8°),NaN,Digital Store 11602.ee.17. (1.),NaN,NaN,English,NaN
4,14602833,Monograph,NaN,NaN,"Anslow, Robert",NaN,person,NaN,"Anslow, Robert [person]","The Defeat of the Spanish Armada, A.D. 1588. A...",...,Elliot Stock,1888,NaN,40 pages (8°),NaN,Digital Store 11602.ee.17. (7.),NaN,NaN,English,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52690,16289058,Monograph,NaN,NaN,"Dickens, Charles",1812-1870,person,NaN,"Dickens, Charles, 1812-1870 [person]",The posthumous papers of the Pickwick Club,...,World's Best Library,NaN,NaN,"xvi, 610 pages, illustrations, 20 cm",823.8,NaN,England--Social life and customs--19th century...,NaN,English,Spine title: The Pickwick papers
52691,16289059,Serial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRUE STORY CLASSICS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,NaN
52692,16289060,Monograph,NaN,NaN,"Wellesley, Dorothy",1889-1956,person,NaN,"Wellesley, Dorothy, 1889-1956 [person]",Early Poems. By M. A [i.e. Dorothy Violet Well...,...,Elkin Mathews,1913,NaN,"vii, 90 pages (8°)",NaN,Digital Store 011649.eee.17,NaN,NaN,English,NaN
52693,16289061,Monograph,NaN,NaN,"A, T. H. E.",NaN,person,NaN,"A, T. H. E. [person]","Of Life and Love [Poems.] By T. H. E. A, write...",...,J. M. Watkins,1924,NaN,89 pages (8°),NaN,Digital Store 011645.e.125,NaN,NaN,English,NaN


In [370]:
df.columns

Index(['BL record ID', 'Type of resource', 'BNB number', 'ISBN', 'Name',
       'Dates associated with name', 'Type of name', 'Role', 'All names',
       'Title', 'Variant titles', 'Series title', 'Number within series',
       'Country of publication', 'Place of publication', 'Publisher',
       'Date of publication', 'Edition', 'Physical description',
       'Dewey classification', 'BL shelfmark', 'Topics', 'Genre', 'Languages',
       'Notes'],
      dtype='object')

### Profiling data automatically 


We can get a rough overview of our dataset using [pandas profiling](https://github.com/pandas-profiling/pandas-profiling)

In [410]:
from pandas_profiling import ProfileReport

profile = ProfileReport(
    df,
    title="Pandas Profiling Report",
    correlations=None,
)

In [411]:
profile.config.interactions.continuous = False

In [413]:
profile.to_notebook_iframe()

This gives us a very useful overview of our data. We'll also produce a quick summary of how often values are missing. 

### Look at coverage 

In [388]:
percent_coverage = df.notna().sum() * 100 / len(df)
percent_coverage_df = pd.DataFrame(
    {"column_name": df.columns, "percent_coverage": percent_coverage}
)

In [399]:
percent_coverage_df.sort_values("percent_coverage", inplace=True, ascending=False)
percent_coverage_df

,column_name,percent_coverage
BL record ID,BL record ID,100.000000
Title,Title,100.000000
Type of resource,Type of resource,100.000000
Languages,Languages,99.889933
Date of publication,Date of publication,99.662207
BL shelfmark,BL shelfmark,99.493311
Place of publication,Place of publication,98.534965
All names,All names,94.189202
Name,Name,90.240061
Type of name,Type of name,90.240061


### Coverage cutoff 

Suggested cut off at 50% i.e. don't include any fields below `Publisher`. After this there is a steep drop off in coverage so these additional fields are likely to be missing too often to be very useful when working 'at scale' which this dataset is intended to support. 

## Looking at fields we want to keep we need to think about 

- How to store? i.e. is it listy or textual
- Is there any tidying to do?
- Do we need to split any out etc. 

## Language

### How many languages to include in the dataset?

We have some books with multiple languages, these are deliniated by `;`. For example

In [ ]:
df[df["Languages"].str.split(";").str.len() > 1]["Languages"].head(5)

170              English ; Latin
187    English ; German ; Hebrew
229             English ; French
529              English ; Latin
692            English ; Italian
Name: Languages, dtype: object

We have a few options for storing this data, including:
- show all languages into a list and store this as a single JSON array
- enumerate languages up-to `n` number and then include no more languages
- only take first language

To help make this decision lets see what percentage of the time we have multiple languages. We can grab examples with more than one language

In [ ]:
df[df["Languages"].str.split(";").str.len() > 1]["Languages"]

170                English ; Latin
187      English ; German ; Hebrew
229               English ; French
529                English ; Latin
692              English ; Italian
                   ...            
49334               German ; Latin
49350               Dutch ; French
49355              Latin ; Russian
49401             French ; Russian
52082              English ; Latin
Name: Languages, Length: 617, dtype: object

In [ ]:
len(df[df["Languages"].str.split(";").str.len() > 1]["Languages"]) / len(df)

0.011708890786602144

Lets see what the distribution of number of languages looks like

In [ ]:
df.Languages.str.split(";").str.len().describe()

count    52637.000000
mean         1.014666
std          0.145131
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          4.000000
Name: Languages, dtype: float64

####  How many more than 3 languages?

In [ ]:
len(df[df.Languages.str.split(";").str.len() > 3])

19

### Suggested approach for storing languages

Since language is an important field for many types of work it justifies a bit more storage space so it's suggested to store all languages listed as separate values. We also have a flag to indicate quickly multiple languages are listed

In [ ]:
{
    "Language_1": "English",
    "Language_2": "French",
    "Language_3": None,
    "Language_4": None,
    "multi_language": True,
}

{'Language_1': 'English',
 'Language_2': 'French',
 'Language_3': None,
 'Language_4': None,
 'multi_language': True}

### Checking languages for duplication?

Confirm how many unique languages we have 

In [ ]:
import numpy as np

In [ ]:
languages = df.Languages.to_list()

In [ ]:
languages = [(None if l is np.nan else l) for l in languages]

In [ ]:
from toolz.itertoolz import concat, frequencies, unique

In [ ]:
languages = list(
    filter(lambda x: x is not None, [(l.split("; ") if l else l) for l in languages])
)

In [ ]:
frequencies(concat(languages))

{'English': 41232,
 'German': 3263,
 'French': 3932,
 'English ': 176,
 'Latin': 275,
 'German ': 166,
 'Hebrew': 2,
 'Italian': 739,
 'Welsh': 2,
 'Danish ': 29,
 'Greek, Ancient (to 1453) ': 2,
 'Greek, Ancient (to 1453)': 1,
 'Spanish': 791,
 'Swedish': 267,
 'Finnish': 14,
 'Dutch': 551,
 'Portuguese': 81,
 'Dutch ': 81,
 'Danish': 230,
 'Italian ': 9,
 'Hungarian': 285,
 'Greek, Modern (1453-)': 56,
 'Russian': 628,
 'Polish': 182,
 'Sanskrit': 1,
 'Cornish ': 1,
 'French ': 200,
 'Irish': 5,
 'Serbian': 13,
 'Czech': 56,
 'Greek, Modern (1453-) ': 31,
 'Norwegian Nynorsk': 3,
 'Russian ': 5,
 'Slovenian': 2,
 'Bulgarian ': 19,
 'Finnish ': 20,
 'Lithuanian': 3,
 'Romanian': 7,
 'Bulgarian': 2,
 'Slovak': 2,
 'Icelandic': 6,
 'Polish ': 6,
 'Latin ': 23,
 'Ukrainian': 1,
 'Indonesian ': 1,
 'Hungarian ': 2,
 'Norwegian': 1,
 'Icelandic ': 1,
 'Scots': 1,
 'Croatian': 3}

We can see some near duplicates

In [ ]:
languages = list(unique(concat(languages)))

In [ ]:
len(languages)

50

If we strip out the white space this seems to deal with this

In [ ]:
set(map(lambda x: x.strip(" "), languages))

{'Bulgarian',
 'Cornish',
 'Croatian',
 'Czech',
 'Danish',
 'Dutch',
 'English',
 'Finnish',
 'French',
 'German',
 'Greek, Ancient (to 1453)',
 'Greek, Modern (1453-)',
 'Hebrew',
 'Hungarian',
 'Icelandic',
 'Indonesian',
 'Irish',
 'Italian',
 'Latin',
 'Lithuanian',
 'Norwegian',
 'Norwegian Nynorsk',
 'Polish',
 'Portuguese',
 'Romanian',
 'Russian',
 'Sanskrit',
 'Scots',
 'Serbian',
 'Slovak',
 'Slovenian',
 'Spanish',
 'Swedish',
 'Ukrainian',
 'Welsh'}

In [ ]:
len(set(map(lambda x: x.strip(" "), languages)))

35

## Date

This is another important field. Most of the time we have a single value which could be cast to a `int` i.e. `1968` but there are some others which include a data range i.e. `1830-1839`. We'll start by looking at the values for each of our dates. 

In [ ]:
df["Date of publication"].value_counts()

1897         1478
1896         1414
1895         1277
1893         1205
1890         1182
             ... 
1879-1885       1
1834-           1
1896-           1
1859-1862       1
1950            1
Name: Date of publication, Length: 458, dtype: int64

Grab dates where there are 5 characters

In [ ]:
df[df["Date of publication"].str.len() == 5]["Date of publication"]

1701     1821-
2990     1886-
3211     1896-
3556     1874-
3557     1829-
4295     1830-
8592     1846-
11783    1819-
16822    1781-
21202    1808-
21203    1872-
21204    1795-
22711    1834-
22724    1864-
24216    1817-
24217    1843-
24218    1839-
24219    1838-
24220    1855-
24222    1876-
24223    1893-
24224    1802-
24226    1887-
24227    1822-
24228    1824-
27200    1825-
29804    1917-
30435    1866-
31595    1853-
31596    1845-
31597    1858-
33393    1853-
34757    1887-
34758    1823-
36030    1805-
36031    1858-
37490    1829-
39569    1897-
43106    1871-
46710    1884-
Name: Date of publication, dtype: object

This could be dealt with by stripping `-` from dates 

### How many dates with ranges?

In [ ]:
df_public[df_public["Date of publication"].str.len() == 9]["Date of publication"]

5085     1878-1879
5097     1872-1873
5100     1892-1925
6569     1849-1850
6570     1890-1891
           ...    
48235    1892-1901
48319    1835-1836
49020    1869-1872
49023    1885-1900
49034    1860-1863
Name: Date of publication, Length: 74, dtype: object

In [ ]:
len(df_public[df_public["Date of publication"].str.len() == 9]["Date of publication"])

74

In [ ]:
df_date_ranges = df_public[df_public["Date of publication"].str.len() > 5]
df_date_ranges = (
    df_date_ranges["Date of publication"].str.split("-", expand=True).astype(int)
)
df_date_ranges

,0,1
5085,1878,1879
5097,1872,1873
5100,1892,1925
6569,1849,1850
6570,1890,1891
...,...,...
48235,1892,1901
48319,1835,1836
49020,1869,1872
49023,1885,1900


In [ ]:
df_date_ranges.diff(axis=1)[1].describe()

count     74.000000
mean      10.081081
std       19.908528
min       -9.000000
25%        1.000000
50%        3.000000
75%       10.000000
max      109.000000
Name: 1, dtype: float64

In [ ]:
df_date_ranges.diff(axis=1)[1].sort_values(ascending=False).head(25)

15178    109
36375     94
44064     89
46382     37
5100      33
43919     25
46354     25
34081     23
37811     22
46209     20
37456     20
49023     15
47585     14
43921     14
36396     14
21802     13
41504     12
42250     11
9208      10
43797     10
44815      9
48235      9
46956      8
36172      7
45485      7
Name: 1, dtype: int64

### Suggested approach

Take mean of values for dates with a range (as we do for the XML) and store 'raw' data values as a string for anyone who wants to parse differently


In [ ]:
{"date": 1850, "raw_date": "1850-"}

{'date': 1850, 'raw_date': '1850-'}

## Country of publication

In [ ]:
df["Country of publication"].unique()

array(['England', 'Ireland', nan, 'England ; United States of America',
       'Scotland', 'Barbados', 'Spain', 'Saint Helena',
       'United States of America', 'Wales', 'France', 'England ; Germany',
       'England ; Scotland', 'Germany', 'Italy', 'England ; Ireland',
       'Canada', 'Northern Ireland', 'Singapore',
       'England ; Northern Ireland', 'Burma', 'Jamaica', 'New Zealand',
       'Malta', 'India', 'Macao', 'Mauritius', 'South Africa',
       'Netherlands', 'Poland', 'England ; Wales',
       'England ; Ireland ; Northern Ireland',
       'England ; Scotland ; United States of America', 'Gibraltar',
       'Ireland ; Northern Ireland', 'England ; India ; Scotland',
       'England ; Germany ; United States of America', 'Russia',
       'Austria', 'England ; United Kingdom', 'Madagascar', 'Switzerland',
       'Hungary', 'England ; South Africa', 'Serbia', 'Australia',
       'Saint-Martin', 'Portugal', 'Romania', 'Austria ; Czech Republic',
       'Slovakia', 'Ukraine

In [ ]:
df[df["Country of publication"].str.split(";").str.len() > 1]["Country of publication"]

11       England ; United States of America
419                       England ; Germany
749                      England ; Scotland
870                       England ; Germany
913                      England ; Scotland
                        ...                
52486                       England ; Wales
52518    England ; United States of America
52558    England ; United States of America
52612                    England ; Scotland
52686                    England ; Scotland
Name: Country of publication, Length: 1102, dtype: object

In [ ]:
len(
    df[df["Country of publication"].str.split(";").str.len() > 1][
        "Country of publication"
    ]
) / len(df)

0.02091280007590853

In [ ]:
df[df["Country of publication"].str.split(";").str.len() > 2]["Country of publication"]

10731             England ; Ireland ; Northern Ireland
12939    England ; Scotland ; United States of America
16659                       England ; India ; Scotland
17512     England ; Germany ; United States of America
28469    England ; Scotland ; United States of America
30137    England ; Scotland ; United States of America
38531                     Barbados ; England ; Ireland
Name: Country of publication, dtype: object

In [ ]:
df[df["Country of publication"].str.split(";").str.len() > 3]["Country of publication"]

Series([], Name: Country of publication, dtype: object)

### Suggested format for Country of Publication

In [ ]:
{
    "Country of publication 1": "England",
    "All Countries of publication": ["England", "Ireland", "Norther Ireland"],
}

{'Country of publication 1': 'England',
 'All Countries of publication': ['England', 'Ireland', 'Norther Ireland']}

## Place of publication

In [ ]:
df["Place of publication"].value_counts()

London                            26743
Paris                              2132
New York                           1179
Edinburgh                          1026
Edinburgh ; London                  524
                                  ...  
Kolozsvártt, Pest                     1
Bielefeld & Leipzig, Altenburg        1
Paris, Evreux                         1
Windham, New Hampshire                1
Willington                            1
Name: Place of publication, Length: 3492, dtype: int64

In [ ]:
df[df["Place of publication"].str.split(";").str.len() > 1]["Place of publication"]

11                        London ; New York
53                          Hereford ; Hull
149                      Cambridge ; London
201                       Calcutta ; London
203                  Bradford ; Wallingford
                        ...                
52517    Greenfield, Massachusetts ; London
52518                     London ; New York
52558                     London ; New York
52612                      Glasgow ; London
52686                    Edinburgh ; London
Name: Place of publication, Length: 2844, dtype: object

In [ ]:
len(
    df[df["Place of publication"].str.split(";").str.len() > 1]["Place of publication"]
) / len(df)

0.05397096498719044

### Suggested storage format

In [ ]:
{
    "Place of publication 1": "London",
    "All Places of publication": ["Bradford", "Wallingford"],
}

{'Place of publication 1': 'London',
 'All Places of publication': ['Bradford', 'Wallingford']}

## All names


In [ ]:
df["All names"].value_counts()

Byron, George Gordon Byron, Baron, 1788-1824 [person]                                                     126
Wood, Henry, Mrs, 1814-1887 [person]                                                                      103
Oliphant, Mrs (Margaret), 1828-1897 [person]                                                               81
Scott, Walter, Sir, 1771-1832 [person]                                                                     78
Great Britain, Hydrographic Department [organisation]                                                      69
                                                                                                         ... 
O'Brien, Augustin P. [person]                                                                               1
O'Bill, Rex [person]                                                                                        1
Oates, C. G. (Charles George) [person] ; Oates, Frank, f.r.g.s [person] ; Rolfe, Robert Allen [person]      1
Oates, C. 

Split into a list on `;`

In [ ]:
df["All names"].str.split(";").values[0]

['More, Hannah, 1745-1833 [person] ', ' Yearsley, Ann, 1753-1806 [person]']

### Suggested storage format

In [ ]:
{
    "All names": [
        "More, Hannah, 1745-1833 [person] ",
        " Yearsley, Ann, 1753-1806 [person]",
    ]
}

{'All names': ['More, Hannah, 1745-1833 [person] ',
  ' Yearsley, Ann, 1753-1806 [person]']}

## Type of resource

In [ ]:
df["Type of resource"].value_counts()

Monograph                     52556
Monographic component part       93
Serial                           46
Name: Type of resource, dtype: int64

## Other overlapping data

- Titles we want to leave alone
- variations in name we want to leave alone 

These might be 'tidied' for some applications but this is left to the end user since this change is destructive and won't always be desired